In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
def get_links(page_link):
    headlines=[]
    stories=[]
    titles=[]
    dates=[]
    r = requests.get(page_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    content = soup.find('div', {'class':'topic_content'})

    
    for article in content.findAll('div', {'class':'news_item'}):
        #get date
        dates.append(article.find('span', {'class':'date'}).text)
        
        a_tag = article.find('a', attrs={'data-ga-action': 'Topic: Story Headline'})
        link = "https://www.democracynow.org"+a_tag['href']
        #get title
        titles.append(a_tag.text)
        if 'headlines' in link:
            headlines.append(link)
        else:
            stories.append(link)
    return headlines, stories, titles, dates

def get_text(page_link, category):
    '''
        Category: Headline or Story
    '''
    article=''
    r=requests.get(page_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    if category == 'headline':
        headline_summary = soup.find('div', {'class':'headline_summary'})
        for p in headline_summary.findAll('p'):
            article = article + "¶" + p.text.replace(u'\xa0', u' ')
    elif category == 'story':
        #story summary
        summary = soup.find('div', {'class':'story_summary'})
        for p in summary.findAll('p'):
            article = article + "¶" + p.text.replace(u'\xa0', u' ')
        #story transcript
        transcript = soup.find('div', {'id':'transcript'})
        #remove strong tags of speaker name in transcript
        if transcript:
            [s.extract() for s in transcript('strong')]
            for p in transcript.findAll('p'):
                article = article + "¶" + p.text.replace(u'\xa0', u' ')
    return article

In [24]:
#scraping from all pages
democracynow_articles=[]
democracynow_titles=[]
democracynow_dates=[]
for i in range(1, 131):
    headlines, stories, titles, dates = get_links("https://www.democracynow.org/topics/climate_change/{}".format(i))
    democracynow_titles.extend(titles)
    democracynow_dates.extend(dates)
    for headline in headlines:
        democracynow_articles.append(get_text(headline, category='headline'))
    for story in stories:
        democracynow_articles.append(get_text(story, category='story'))

In [25]:
len(democracynow_dates)

1300

In [54]:
articles_shorter=[]
for article in democracynow_articles:
    if len(article)<=30000:
        articles_shorter.append(article)

In [55]:
len(articles_shorter)

1249

In [26]:
democracynow_articles[650]

'¶Activists from the group Greenpeace hung a massive, 70-by-35-foot banner reading ”RESIST” from a crane only blocks from the White House Wednesday. This is Greenpeace’s Nancy Hernandez.¶Nancy Hernandez: “I’m standing here on top of a crane 300 feet in the air over the White House, where we have just deployed a banner. And this banner, it is a message to this administration. But more than that, this is a hand-painted love letter to you. This is a message to the people.”'

In [28]:
df = pd.DataFrame(democracynow_articles, columns=['content'])
df['title']=democracynow_titles
df['author']=['NaN']*len(df.index)
df['date']=democracynow_dates
df['denial?'] = [0]*len(df.index)
cols=['title','author','date','content','denial?']
df=df[cols]
df = df.drop_duplicates()
df.to_csv('../../data/left/data_democracynow_articles.csv', index=False)